# [WUM] PD4 - Mateusz Polakowski
Czwarta praca domowa opiera się na zbadaniu działania i celności algorytmu *Support Vector Machine* (*SVM*). Zrobimy to korzystając z dwóch zbiorów: *apartments* z R-owego pakietu *DALEX* oraz *vehicle* zaciągniętego z *OpenML*. Wybór taki można wytłumaczyć między innymi tym, że *apartments* jest problemem regresji, natomiast *vehicle*, dla odmiany, klasyfikacji. Spojrzymy na oba problemy symultanicznie, aby wysnuwać wnioski z kolejnych etapów ulepszających predykcję.

## Załadowanie potrzebnych pakietów i definicja funkcji

In [164]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing.data import StandardScaler
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import sklearn.metrics
import openml
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

def evaluate_model(model, df_subsets, task):
    """
        Uczy model, a następnie zwraca wynik ewaluacji, w zależności od zadania:
        - regresja - RMSE (root_mean_squared_error)
        - klasyfikacja - ACC (accuracy)
    """
    model.fit(df_subsets[0], df_subsets[2]) # X_train,y_train
    
    if task == "classification":
        return np.round(sklearn.metrics.accuracy_score(df_subsets[3], 
                                                       model.predict(df_subsets[1])), # y_test, X_test
                        4)
    elif task == "regression":
        return np.round(sqrt(sklearn.metrics.mean_squared_error(df_subsets[3], 
                                                                model.predict(df_subsets[1]))), # y_test, X_test
                        4)

## Wczytanie danych
### vehicle

In [139]:
dataset = openml.datasets.get_dataset(54)
X, y, categorical, names = dataset.get_data(
    target=dataset.default_target_attribute,
    return_categorical_indicator=True,
    return_attribute_names=True,
    include_ignore_attributes=True
)

vals = {}
for i, name in enumerate(names):
    vals[name] = X[:, i]
vals[dataset.default_target_attribute] = y
df_vehicle = pd.DataFrame(vals)
X_vehicle, y_vehicle = df_vehicle.drop("Class", axis=1), df_vehicle.loc[:, "Class"]
subsets_vehicle = train_test_split(X_vehicle, y_vehicle,
                                   test_size=0.15,
                                   random_state=1410)

df_vehicle.head()

,COMPACTNESS,CIRCULARITY,DISTANCE_CIRCULARITY,RADIUS_RATIO,PR.AXIS_ASPECT_RATIO,MAX.LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR.AXIS_RECTANGULARITY,MAX.LENGTH_RECTANGULARITY,SCALED_VARIANCE_MAJOR,SCALED_VARIANCE_MINOR,SCALED_RADIUS_OF_GYRATION,SKEWNESS_ABOUT_MAJOR,SKEWNESS_ABOUT_MINOR,KURTOSIS_ABOUT_MAJOR,KURTOSIS_ABOUT_MINOR,HOLLOWS_RATIO,Class
0,95.0,48.0,83.0,178.0,72.0,10.0,162.0,42.0,20.0,159.0,176.0,379.0,184.0,70.0,6.0,16.0,187.0,197.0,3
1,91.0,41.0,84.0,141.0,57.0,9.0,149.0,45.0,19.0,143.0,170.0,330.0,158.0,72.0,9.0,14.0,189.0,199.0,3
2,104.0,50.0,106.0,209.0,66.0,10.0,207.0,32.0,23.0,158.0,223.0,635.0,220.0,73.0,14.0,9.0,188.0,196.0,1
3,93.0,41.0,82.0,159.0,63.0,9.0,144.0,46.0,19.0,143.0,160.0,309.0,127.0,63.0,6.0,10.0,199.0,207.0,3
4,85.0,44.0,70.0,205.0,103.0,52.0,149.0,45.0,19.0,144.0,241.0,325.0,188.0,127.0,9.0,11.0,180.0,183.0,2


### apartments

In [149]:
df_apartments = pd.read_csv("./../../../../Datasets/PD4 - SVM/apartments.csv").iloc[:, 1:]
X_apartments, y_apartments = df_apartments.drop("m2.price", axis=1), df_apartments['m2.price']
subsets_apartments = train_test_split(X_apartments, y_apartments,
                                      test_size=0.15,
                                      random_state=1410)
df_apartments.head()

,m2.price,construction.year,surface,floor,no.rooms,district
0,5897,1953,25,3,1,Srodmiescie
1,1818,1992,143,9,5,Bielany
2,3643,1937,56,1,2,Praga
3,3517,1995,93,7,3,Ochota
4,3013,1992,144,6,5,Mokotow


## Predykcja na domyślnych modelach

### vehicle

In [160]:
evaluate_model(SVC(), 
               subsets_vehicle, 
               "classification")

0.2677

### apartments

In [217]:
# OneHotEncoding użyty do zakodowania dzielnic
evaluate_model(Pipeline([("OneHotEncoder", OneHotEncoder()),
                         ("SVR", SVR())]), 
               subsets_apartments, 
               "regression")

919.5456

Jak widać, powyższe wyniki nie są wcale satysfakcjonujące, a wręcz zatrważająco słabe. $27\%$ celności i średnio tysiąc błędu w estymacji ceny za metr kwadratowy mieszkania oscylującej w okolicach kilku tysięcy to faktycznie kiepski wynik. Spróbujmy najpierw przyjrzeć się standaryzacji danych o której mowa w [tym artykule](http://pyml.sourceforge.net/doc/howto.pdf):

### vehicle

In [162]:
evaluate_model(Pipeline([('StandardScaler', StandardScaler()), 
                         ('SVC', SVC())]),
               subsets_vehicle,
               "classification")

0.7402

### apartments

In [163]:
evaluate_model(Pipeline([("OneHotEncoder", OneHotEncoder()),
                         ("StandardScaler", StandardScaler()),
                         ("SVR", SVR())]), 
               subsets_apartments, 
               "regression")

894.8422

W modelu opartym na danych ze zbioru *vehicle* widać ogromną różnicę, natomiast w przypadku *apartments* jedynie subtelną. Został jeszcze chwyt, który w tej pracy przećwiczymy, mianowicie użycie algorytmu z pakietu *model_selection*, a mianowicie *RandomizedSearchCV*. Jest to analogia do algorytmu *GridSearchCV* w szukaniu najbardziej optymalnych hiperparametrów. Różnią się tym, żeby zamiast sprawdzać wszystkich możliwości, które mogą się namnożyć bardzo szybko, algorytm sprawdza tylko określoną liczbe iteracji wskazanych *możliwości* hiperparametrów. Nawiązując do artykułu oraz możliwości jądra *RBF* w algorytmie *SVM*, zastosujemy wyszukiwanie najlepszych wartości hiperparametrów C, *gamma* oraz *shrinking*:

In [172]:
param_dist_vehicle = {
    "SVC__C": [0.5] + list(np.arange(1, 100, 5)),
    "SVC__gamma": [0.01, 0.03,0.05, 0.07, 0.1, 0.3, 0.5, 0.7] + list(np.arange(1, 100, 10)),
    "SVC__shrinking": [True, False]
}

param_dist_apartments = {
    "SVR__C": [0.5] + list(np.arange(1, 100, 5)),
    "SVR__gamma": [0.01, 0.03,0.05, 0.07, 0.1, 0.3, 0.5, 0.7] + list(np.arange(1, 100, 10)),
    "SVR__shrinking": [True, False]
}

### vehicle

In [177]:
random_search_vehicle = RandomizedSearchCV(Pipeline([('StandardScaler', StandardScaler()), 
                                                     ('SVC', SVC())]), 
                                           param_distributions=param_dist_vehicle, 
                                           n_iter=50, 
                                           cv=5, 
                                           scoring="accuracy")
random_search_vehicle.fit(X_vehicle, y_vehicle)
print("Accuracy: ", np.round(random_search_vehicle.best_score_, 4), "\n",
      "Best params: ", random_search_vehicle.best_params_, sep="")

Accuracy: 0.8475
Best params: {'SVC__shrinking': True, 'SVC__gamma': 0.05, 'SVC__C': 56}


### apartments

In [182]:
random_search_apartments = RandomizedSearchCV(Pipeline([("OneHotEncoder", OneHotEncoder()),
                                                        ("StandardScaler", StandardScaler()),
                                                        ("SVR", SVR())]), 
                                           param_distributions=param_dist_apartments, 
                                           n_iter=50, 
                                           cv=5, 
                                           scoring="neg_mean_squared_error")
random_search_apartments.fit(X_apartments, y_apartments)
print("Root mean squared error: ", np.round(sqrt(-random_search_apartments.best_score_), 2), "\n",
      "Best params: ", random_search_apartments.best_params_, sep="")

Root mean squared error: 291.64
Best params: {'SVR__shrinking': False, 'SVR__gamma': 0.05, 'SVR__C': 81}


Takie wyniki można uznać już za satysfakcjonujące - celność na zbiorze *vehicle* w okolicach rekordowej na serwisie *OpenML*, a średnia estymacja ceny zmniejszona z tysiąca do 300. Jak widać powyżej obie techniki, standaryzacja zmiennych oraz dobieranie hiperparametrów, znacząco mogą podnieść jakość modelu.